In [78]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community langchainhub

In [79]:
question = "What is the author's thought about RAG?"

In [80]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [81]:
def contains_word(docs, word):
    for doc in docs:
        if word in doc.page_content:
            return True
    return False

In [82]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://applied-llms.org/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("content")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [83]:
import sys

def relevance_checker() :
    if contains_word(docs, "Author"):
        print("[relevance check] : OK")

    docs_mmr = vectorstore.max_marginal_relevance_search(question, k=2, fetch_k=3)
    print(docs_mmr[0].page_content[:100])
    print(docs_mmr[1].page_content[:100])
    if docs_mmr[0].page_content[:100] == docs_mmr[1].page_content[:100]:
        print("[relevance check] : OK")
        return True
    return False
    
if not relevance_checker():
    print("[relevance check] : No")
    sys.exit()


[relevance check] : OK
1.2.1 RAG is only as good as the retrieved documents’ relevance, density, and detail
The quality of 
1.2.1 RAG is only as good as the retrieved documents’ relevance, density, and detail
The quality of 
[relevance check] : OK


In [84]:

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
)


In [85]:
answer = rag_chain.invoke(question)
limit = 0

def hallucination_checker(last_answer):
    if not last_answer:
        print("[hallucination check] : last_answer is empty")
        return False

    answer_for_checking_hallucination = rag_chain.invoke("last answer : " + last_answer + "\nWhat do you think this last answer means RAG is related with density? please answer yes or no")
    print("[answer for hallucination check] :" + answer_for_checking_hallucination)
    
    if "Yes." in answer_for_checking_hallucination:
        print("[hallucination check] : okay")
        print(last_answer)
        return True
    return False


while not hallucination_checker(answer) and limit < 10:
    limit += 1
    print(limit)




[answer for hallucination check] :Yes. The context indicates that the effectiveness of the RAG system depends on the density of the retrieved documents, along with relevance and detail. Therefore, density is indeed related to RAG.
[hallucination check] : okay
The author believes that the effectiveness of RAG (Retrieval-Augmented Generation) is heavily reliant on the relevance, density, and detail of the retrieved documents. The quality of RAG's output is determined by these factors, with relevance being a key metric assessed through ranking metrics like Mean Reciprocal Rank (MRR) and Normalized Discounted Cumulative Gain (NDCG). Thus, a well-functioning RAG system must effectively rank relevant documents higher than irrelevant ones.
